In [1]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 46.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=839e2d2643cf81c749fc7d94f861e8a6e0cc21c22bab5fe454119626dd98020c
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


# 1. 데이터 로드

In [2]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


# 2. SBERT 임베딩

문장 임베딩을 얻기 위해 사전 훈련된 BERT를 로드. 

In [4]:
# 한국어 포함 다국어 모델
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

- 100가지 언어를 지원(한국어 포함)하는 다국어 BERT BASE 모델로 SNLI 데이터를 학습 후 STS-B 데이터로 학습되었으며, 문장 표현을 얻기 위해서는 평균 풀링(mean-tokens)을 사용했다.
- NLI 데이터를 학습 후에 STS 데이터로 추가 파인 튜닝한 모델이라는 의미입니다.

모든 질문열에 대한 문장 임베딩 값을 구한 후 embeding이라는 새로운 열에 저장

In [5]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

# 3. 챗봇 테스트

`cos_sim(A,B)` 두 개의 벡터로부터 코사인 유사도를 구하는 함수 

In [6]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

`return_answer()` : 임의의 질문이 들엉오면 해당 질문의 문장 임베딩 값과 챗봇 데이터의 임베딩 열에 저장된 모든 질문 샘플들의 문장 임베딩 값들을 전부 비교하여 코사인 유사도 값이 가장 높은 질문 샘플을 찾아낸다. 그리고 해당 질문 샘플과 짝이 되는 답변 샘플을 리턴한다.

In [8]:
def return_answer(question):
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

In [9]:
return_answer('나랑 커피먹을래?')

'카페인이 필요한 시간인가 봐요.'

In [10]:
return_answer('반가워')

'저도 반가워요.'

In [11]:
return_answer('사랑해')

'상대방에게 전해보세요.'

In [12]:
return_answer('너는 누구니?')

'저는 위로봇입니다.'

In [13]:
return_answer('너무 짜증나')

'짜증날 땐 짜장면'

In [14]:
return_answer('화가납니다')

'화를 참는 연습을 해보세요.'

In [15]:
return_answer('나랑 놀자')

'지금 그러고 있어요.'

In [16]:
return_answer('나랑 게임하자')

'같이 놀아요.'

In [17]:
return_answer('출근하기싫어')

'씻고 푹 쉬세요.'

In [18]:
return_answer('여행가고싶다')

'이김에 떠나보세요.'

In [19]:
return_answer('너 말 잘한다')

'그런 사람이 있으면 저 좀 소개시켜주세요.'